# Importamos los módulos necesarios para el proyecto

In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plot
import src.funciones as fun

In [2]:
!pip3 install regex

## Cargar dataset

In [3]:
df = pd.read_csv("data/attacks.csv",encoding = "ISO-8859-1")
df.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


# DATA CLEANING

## Filtrar columnas a usar

Creamos un nuevo dafaframe con las columnas que son útiles para las hipótesis.

In [4]:
df = df[["Year","Type","Country","Activity","Sex ","Injury","Fatal (Y/N)"]]

Renombramos la columna Sex y Fatal

In [5]:
df = df.rename(columns = {"Sex ": "sex","Fatal (Y/N)":"Fatal"})


## Limpieza de datos

Comprobar duplicados del DF

In [6]:
print(f"Hay un total de: {df.duplicated().sum()}")


Hay un total de: 19511


Eliminamos los duplicados

In [7]:
df = df.drop_duplicates()
df.head()

,Year,Type,Country,Activity,sex,Injury,Fatal
0,2018.0,Boating,USA,Paddling,F,"No injury to occupant, outrigger canoe and pad...",N
1,2018.0,Unprovoked,USA,Standing,F,Minor injury to left thigh,N
2,2018.0,Invalid,USA,Surfing,M,Injury to left lower leg from surfboard skeg,N
3,2018.0,Unprovoked,AUSTRALIA,Surfing,M,Minor injury to lower leg,N
4,2018.0,Provoked,MEXICO,Free diving,M,Lacerations to leg & hand shark PROVOKED INCIDENT,N


## Limpieza de columnas

Miramos los valores nulos de las columnas a utilizar, para cambiarlos después por otro valor sin tener que eliminarlos

In [8]:
df.isnull().sum()

Year          3
Type          5
Country      51
Activity    536
sex         561
Injury       28
Fatal       537
dtype: int64

In [9]:
df.Activity = df.Activity.fillna("UNKNOWN") #Nueva categoria para las actividades que desconocemos
df.Year = df.Year.fillna("0") #Son 3 valores nulos que cambio a 0, y que no afectará a la hipótesis
df.Country = df.Country.fillna("UNKNOWN") #Nueva categoria para los paises que desconocemos
df.sex = df.sex.fillna("UNKNOWN") #Nueva categoria para los sexos que desconocemos
df.Fatal = df.Fatal.fillna("UNKNOWN") #Nueva categoria para los sexos que desconocemos

Resultado final de los nulos

In [10]:
df.isnull().sum()

Year         0
Type         5
Country      0
Activity     0
sex          0
Injury      28
Fatal        0
dtype: int64

### Limpieza Year
Casteamos el año a integer para trabajar mejor la columna

In [11]:
df["Year"] = df["Year"].astype(int)

In [12]:
df.head()

,Year,Type,Country,Activity,sex,Injury,Fatal
0,2018,Boating,USA,Paddling,F,"No injury to occupant, outrigger canoe and pad...",N
1,2018,Unprovoked,USA,Standing,F,Minor injury to left thigh,N
2,2018,Invalid,USA,Surfing,M,Injury to left lower leg from surfboard skeg,N
3,2018,Unprovoked,AUSTRALIA,Surfing,M,Minor injury to lower leg,N
4,2018,Provoked,MEXICO,Free diving,M,Lacerations to leg & hand shark PROVOKED INCIDENT,N


### Limpieza country

In [13]:
pd.unique(df.Country)

array(['USA', 'AUSTRALIA', 'MEXICO', 'BRAZIL', 'ENGLAND', 'SOUTH AFRICA',
       'THAILAND', 'COSTA RICA', 'MALDIVES', 'BAHAMAS', 'NEW CALEDONIA',
       'ECUADOR', 'MALAYSIA', 'LIBYA', 'UNKNOWN', 'CUBA', 'MAURITIUS',
       'NEW ZEALAND', 'SPAIN', 'SAMOA', 'SOLOMON ISLANDS', 'JAPAN',
       'EGYPT', 'ST HELENA, British overseas territory', 'COMOROS',
       'REUNION', 'FRENCH POLYNESIA', 'UNITED KINGDOM',
       'UNITED ARAB EMIRATES', 'PHILIPPINES', 'INDONESIA', 'CHINA',
       'COLUMBIA', 'CAPE VERDE', 'Fiji', 'DOMINICAN REPUBLIC',
       'CAYMAN ISLANDS', 'ARUBA', 'MOZAMBIQUE', 'FIJI', 'PUERTO RICO',
       'ITALY', 'ATLANTIC OCEAN', 'GREECE', 'ST. MARTIN', 'FRANCE',
       'PAPUA NEW GUINEA', 'TRINIDAD & TOBAGO', 'KIRIBATI', 'ISRAEL',
       'DIEGO GARCIA', 'TAIWAN', 'JAMAICA', 'PALESTINIAN TERRITORIES',
       'GUAM', 'SEYCHELLES', 'BELIZE', 'NIGERIA', 'TONGA', 'SCOTLAND',
       'CANADA', 'CROATIA', 'SAUDI ARABIA', 'CHILE', 'ANTIGUA', 'KENYA',
       'RUSSIA', 'TURKS & CAICOS', 

Renombramos algunos paises

In [14]:
df.Country = df.Country.replace({"Seychelles" : "SEYCHELLES", "Fiji" : "FIJI", "Sierra Leone" : "SIERRA LEONE", "RED SEA?" : "RED SEA", "ASIA?": "ASIA"})

### Limpieza Activity

In [15]:
pattern = [r'.*?\bswimming\b.*“,r’.*?\bsurfing\b.*‘, r’.*?\bpaddling\b.*‘, r’.*?\bfishing\b.*‘, r’.*?\bspearfishing\b.*‘, r’.*?\bwading\b.*‘, r’.*?\bscuba\b\s+\bdiving\b.*‘, r’.*?\bsnorkeling\b.*‘, r’.*?\bwalking\b.*‘, r’.*?\bfree\b\s+\bdiving\b.*‘, r’.*?\bbody\b\s+\bboarding\b.*’']
replace = ["swimming","surfing","paddling","fishing","spearfishing","wading","scubadiving","snorkeling","walking","free_diving","body_boarding"]


In [16]:
def reg_activity(df, patron, new_activity):
    return df.Activity.str.lower().replace(patron, new_activity, regex = True)

In [17]:
def lista_coincidencias (patron, string):
    actividad = re.search(patron, string)
    return actividad

### Limpieza Fatal
Agruparemos todos los valores en 3 categorias :
- NO 
- YES 
- UNKNOWN

Comprobamos con un count los valores que tiene la columna

In [18]:
df["Fatal"].value_counts()


N          4246
Y          1350
UNKNOWN     605
 N            7
N             1
2017          1
M             1
y             1
Name: Fatal, dtype: int64

Aplicamos la función definida "fatal" importada anteriormente

In [19]:
df["Fatal"] = df.Fatal.apply(fun.fatal)

Comprobamos que la función es correcta y agrupa todo en 3 categorías

In [20]:
df["Fatal"].value_counts()

NO         4254
YES        1351
UNKNOWN     607
Name: Fatal, dtype: int64

### Limpieza Sex
Agruparemos todos los valores en 3 categorias :
- MALE 
- FEMALE
- UNKNOWN

Comprobamos con un unique que valores son los que hay en la columna

In [27]:
pd.unique(df.sex)

array(['FEMALE', 'MALE', 'UNKNOWN'], dtype=object)

Aplicamos la función definida "sex" importada anteriormente

In [28]:
df["sex"] = df.sex.apply(fun.sex)

Comprobamos que la función es correcta y agrupa todo en 3 categorias

In [29]:
df.sex.value_counts()

MALE       5014
FEMALE      633
UNKNOWN     565
Name: sex, dtype: int64

In [30]:
df.to_csv("data/attacks_clean.csv")